# Toy example for Radix-based Benders

In [1]:
SCALE_CROWDING= False
PREVENT_ZERO  = True

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f9b7010aef0>)


### Try optimizing using radix for one condition first

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound0 = 0.001
crowding_bound  = crowding_bound0
not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [6]:
opt = Optimizer(ijomc)
gap = opt.add_duality_gap_constraint(INF=1e3)

In [7]:
gap.optimize(solver='gurobi')

<Solution 1.75 at 0x7f9bb403b750>

In [8]:
import numpy as np

radix = 2.
powers = np.arange(-3,4)
print(powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print(digits)

# Discretize crowding coefficients into radix
crowding_p = gap.metabolites.crowding

var_cons_dict = {}
for rxn_p in crowding_p.reactions:
    # Get the coefficient in the dual
    var_d = gap.reactions.wa_crowding
    cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]
    a0 = rxn_p.metabolites[crowding_p]    
    var_cons_dict[rxn_p.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

[-3 -2 -1  0  1  2  3]
[1.0]


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [9]:
#%lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

In [10]:
%%time
opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)

CPU times: user 129 ms, sys: 13.8 ms, total: 142 ms
Wall time: 131 ms


[1.0]

In [11]:
### Try scaling
if SCALE_CROWDING:
    crowding = gap.metabolites.crowding
    bound0  = crowding._bound
    f_scale = crowding._bound
    crowding._bound = bound0 / f_scale
    for rxn in crowding.reactions:
        rxn._metabolites[crowding] = rxn._metabolites[crowding]/f_scale

In [12]:
# for rxn in gap.reactions:
#     rxn.objective_coefficient = 0.
# for rxn in gap.reactions.query('binary'):
#     rxn.objective_coefficient = 1.
    
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.

for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = gap.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                #print('Preferring a0')
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = 1.

In [13]:
N_CONDS = 1

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r.id for r in ijomc.reactions.query('EX_')]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [14]:
mu_meas = 0.74

F_TOL = 0.02
rxn_mu = gap.reactions.BIOMASS_Ecoli_core_w_GAM
rxn_mu.lower_bound = mu_meas*(1-F_TOL)
rxn_mu.upper_bound = mu_meas*(1+F_TOL)
print('Initial mu_crowd=%g. Fitting within %g%% of measured: %g <= mu <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, rxn_mu.upper_bound))

    
from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

Initial mu_crowd=0.873922. Fitting within 2% of measured: 0.7252 <= mu <= 0.7548


In [15]:
milp.Params.FeasibilityTol = 1e-9
milp.Params.OptimalityTol = 1e-9
milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 3241 rows, 1366 columns and 8628 nonzeros
Variable types: 1030 continuous, 336 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-01, 1e+03]
  RHS range        [1e-03, 1e+03]
Presolve removed 935 rows and 127 columns
Presolve time: 0.02s
Presolved: 2306 rows, 1239 columns, 6871 nonzeros
Variable types: 903 continuous, 336 integer (336 binary)

Root relaxation: objective 4.676603e-01, 1259 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.46766    0    5          -    0.46766      -     -    0s
     0     0    0.47707    0    5          -    0.47707      -     -    0s
     0     0    0.75120    0    5          -    0.75120      -     -    0s
     0     2    0.7

In [16]:
#gap.optimize('minimize')
rxn_mu =gap.reactions.BIOMASS_Ecoli_core_w_GAM 
#muopt = rxn_mu
sol = gurobi_solver.format_solution(milp, gap)
muopt = sol.x_dict[rxn_mu.id]
yopt = [sol.x_dict[rxn.id] for rxn in gap.reactions.query('binary_')]
sum(yopt)
print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))
print('Number of non-zero binaries: %g' % sum(yopt))

Initial mu_crowd=0.873922. Fitted within 2%: 0.7252 <= 0.750318 <= 0.7548
Number of non-zero binaries: 50


In [17]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol.x_dict[yid]
            if abs(y)>1e-10:
                print yid, y
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

binary_G6PDH2r03 1.0
binary_AKGDH03 1.0
binary_ME203 1.0
binary_MALS03 1.0
binary_GLUN03 1.0
binary_ME103 1.0
binary_PGI03 1.0
binary_GND03 1.0
binary_ACKr03 1.0
binary_GLNS03 1.0
binary_FUM03 1.0
binary_SUCDi03 1.0
binary_PPC03 1.0
binary_MDH03 1.0
binary_GLUDy03 1.0
binary_GLUSy03 1.0
binary_PGL03 1.0
binary_PGM03 1.0
binary_ACALD03 1.0
binary_PGK03 1.0
binary_ADK103 1.0
binary_PPS03 1.0
binary_PTAr03 1.0
binary_RPE03 1.0
binary_ALCD2x03 1.0
binary_SUCOAS03 1.0
binary_TALA03 1.0
binary_NADTRHD03 1.0
binary_ICDHyr03 1.0
binary_GAPD03 1.0
binary_GAPD05 1.0
binary_GAPD06 1.0
binary_ICL03 1.0
binary_TPI03 1.0
binary_ENO03 1.0
binary_ACONTa03 1.0
binary_RPI03 1.0
binary_ACONTb03 1.0
binary_PDH03 1.0
binary_CS03 1.0
binary_TKT203 1.0
binary_FBP03 1.0
binary_PPCK03 1.0
binary_TKT103 1.0
binary_PFK03 1.0
binary_PFL03 1.0
binary_FBA03 1.0
binary_PYK03 1.0
binary_LDH_D03 1.0
binary_FRD703 1.0


In [18]:
kfit_dict1 = dict(kfit_dict)

In [19]:
'Binaries per keff:', len(gap.reactions.query('binary')) / len(kfit_dict)

('Binaries per keff:', 7)

In [20]:
len(var_cons_dict)

48

In [21]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

[(u'G6PDH2r', 4.273504273504274e-06),
 (u'AKGDH', 4.273504273504274e-06),
 (u'ME2', 4.273504273504274e-06),
 (u'PGK', 4.273504273504274e-06),
 (u'GLUN', 4.273504273504274e-06),
 (u'ME1', 4.273504273504274e-06),
 (u'GND', 4.273504273504274e-06),
 (u'ACKr', 4.273504273504274e-06),
 (u'GLNS', 4.273504273504274e-06),
 (u'ADK1', 4.273504273504274e-06),
 (u'SUCDi', 4.273504273504274e-06),
 (u'PPC', 4.273504273504274e-06),
 (u'MDH', 4.273504273504274e-06),
 (u'FUM', 4.273504273504274e-06),
 (u'GLUDy', 4.273504273504274e-06),
 (u'GLUSy', 4.273504273504274e-06),
 (u'PGL', 4.273504273504274e-06),
 (u'PGM', 4.273504273504274e-06),
 (u'ACALD', 4.273504273504274e-06),
 (u'MALS', 4.273504273504274e-06),
 (u'PGI', 4.273504273504274e-06),
 (u'PPS', 4.273504273504274e-06),
 (u'PTAr', 4.273504273504274e-06),
 (u'RPE', 4.273504273504274e-06),
 (u'ALCD2x', 4.273504273504274e-06),
 (u'SUCOAS', 4.273504273504274e-06),
 (u'TALA', 4.273504273504274e-06),
 (u'NADTRHD', 4.273504273504274e-06),
 (u'ICDHyr', 4.27

In [22]:
kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
kfit_changed

Changed keffs: 1/48


[(u'GAPD', 5.555555555555556e-05, 12.0)]

### Plug back in to be sure

In [23]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding = Constraint('crowding')
crowding._bound = crowding_bound0
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

ijofit.optimize()
ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x

'Unoptimized: %g. Fitted: %g' % (mu_crowd0, ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x)

'Unoptimized: 0.873922. Fitted: 0.750318'

# Try using Benders

In [24]:
decomp = Decomposer(gap, objective_sense='minimize')

In [25]:
master,sub = decomp.benders_decomp()

master.Params.presolve = 0
sub.Params.presolve = 0

master.Params.FeasibilityTol = 1e-9
master.Params.OptimalityTol = 1e-9

sub.Params.Method = 0
sub.Params.OutputFlag = 0

sub.Params.FeasibilityTol = 1e-9
sub.Params.OptimalityTol = 1e-9

master._verbosity = 0
master._precision_sub = 'gurobi' #'dq'
master.Params.OutputFlag=0

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Parameter presolve unchanged
   Value: 0  Min: -1  Max: 2  Default: -1
Changed value of parameter presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter FeasibilityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Changed value of parameter OptimalityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1


In [26]:
# ybad = np.array([0. for y in decomp._ys])
# psub = decomp.make_sub_primal(ybad)
# psub.model.Params.InfUnbdInfo=1
# psub.model.optimize()

In [27]:
# ### DEBUG ****************
# # Test an unbounded subproblem using a yopt that is known to be infeasible for primal
# ybad = np.array([0. for y in decomp._ys])
# decomp.update_subobj(ybad)
# sub.model.Params.ModelSense = GRB.MAXIMIZE
# sub.optimize()
# print('Status: %s' % sub.model.Status)
# print('Status is unbounded: %s' % (sub.model.Status==GRB.UNBOUNDED))
# print('Obj val: %s' %sub.ObjVal)

In [28]:
WARM_START_BENDERS = False
# **** DEBUG: warm-start with known solution ****
if WARM_START_BENDERS:
    for yj in master.getVars():
        if 'binary' in yj.VarName:
            yj.Start = sol.x_dict[yj.VarName]
            if sol.x_dict[yj.VarName] > 1e-10:
                print((yj, sol.x_dict[yj.VarName]))

In [29]:
master._verbosity=0
master.Params.OutputFlag = 1
%time master.optimize(cb_benders)

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 385 rows, 337 columns and 961 nonzeros
Variable types: 1 continuous, 336 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [9e-01, 1e+00]
Variable types: 1 continuous, 336 integer (336 binary)

Root relaxation: objective 1.126606e-01, 57 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.11266    0   31          -    0.11266      -     -    0s
     0     0    1.00000    0    4          -    1.00000      -     -    0s
     0     0    1.00000    0    6          -    1.00000      -     -    0s
     0     0    1.00000    0    6          -    1.00000      -     -    0s
     0     0    1.00000    0    2          -    1.00000      -     

In [30]:
master.ObjVal

1.9999999999999978

In [31]:
sub.ObjVal

-0.0

In [32]:
yoptB = np.array([y.X for y in decomp._ys])
sum(yoptB)

50.0

In [33]:
mu_id = 'BIOMASS_Ecoli_core_w_GAM'

cons_mu = sub.model.getConstrByName(str(mu_id))
cons_mu.Pi

0.7548

In [34]:
sol_master = {v.VarName:v.X for v in master.getVars()}

In [35]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
            if abs(y)>1e-10:
                print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

binary_G6PDH2r03. Value=1.0. Power=0. Digit=1
binary_AKGDH03. Value=1.0. Power=0. Digit=1
binary_ME203. Value=1.0. Power=0. Digit=1
binary_MALS03. Value=1.0. Power=0. Digit=1
binary_GLUN03. Value=1.0. Power=0. Digit=1
binary_ME103. Value=1.0. Power=0. Digit=1
binary_PGI03. Value=1.0. Power=0. Digit=1
binary_GND03. Value=1.0. Power=0. Digit=1
binary_ACKr03. Value=1.0. Power=0. Digit=1
binary_GLNS03. Value=1.0. Power=0. Digit=1
binary_FUM03. Value=1.0. Power=0. Digit=1
binary_SUCDi03. Value=1.0. Power=0. Digit=1
binary_PPC03. Value=1.0. Power=0. Digit=1
binary_MDH03. Value=1.0. Power=0. Digit=1
binary_GLUDy03. Value=1.0. Power=0. Digit=1
binary_GLUSy03. Value=1.0. Power=0. Digit=1
binary_PGL03. Value=1.0. Power=0. Digit=1
binary_PGM03. Value=1.0. Power=0. Digit=1
binary_ACALD03. Value=1.0. Power=0. Digit=1
binary_PGK03. Value=1.0. Power=0. Digit=1
binary_ADK103. Value=1.0. Power=0. Digit=1
binary_PPS03. Value=1.0. Power=0. Digit=1
binary_PTAr03. Value=1.0. Power=0. Digit=1
binary_RPE03. 

In [36]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

[(u'G6PDH2r', 4.273504273504274e-06),
 (u'AKGDH', 4.273504273504274e-06),
 (u'ME2', 4.273504273504274e-06),
 (u'PGK', 4.273504273504274e-06),
 (u'GLUN', 4.273504273504274e-06),
 (u'ME1', 4.273504273504274e-06),
 (u'GND', 4.273504273504274e-06),
 (u'ACKr', 4.273504273504274e-06),
 (u'GLNS', 4.273504273504274e-06),
 (u'ADK1', 4.273504273504274e-06),
 (u'SUCDi', 4.273504273504274e-06),
 (u'PPC', 4.273504273504274e-06),
 (u'MDH', 4.273504273504274e-06),
 (u'FUM', 4.273504273504274e-06),
 (u'GLUDy', 4.273504273504274e-06),
 (u'GLUSy', 4.273504273504274e-06),
 (u'PGL', 4.273504273504274e-06),
 (u'PGM', 4.273504273504274e-06),
 (u'ACALD', 4.273504273504274e-06),
 (u'MALS', 4.273504273504274e-06),
 (u'PGI', 4.273504273504274e-06),
 (u'PPS', 4.273504273504274e-06),
 (u'PTAr', 4.273504273504274e-06),
 (u'RPE', 4.273504273504274e-06),
 (u'ALCD2x', 4.273504273504274e-06),
 (u'SUCOAS', 4.273504273504274e-06),
 (u'TALA', 4.273504273504274e-06),
 (u'NADTRHD', 4.273504273504274e-06),
 (u'ICDHyr', 4.27

In [37]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding = Constraint('crowding')
crowding._bound = crowding_bound0
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

ijofit.optimize()
ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x

'Unoptimized: %g. Fitted: %g' % (mu_crowd0, ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x)

'Unoptimized: 0.873922. Fitted: 0.750318'